In [8]:
%autosave 0

Autosave disabled


In [14]:
!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

Defaulting to user installation because normal site-packages is not writeable
  Using cached grpcio-1.42.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.0 MB)
  Using cached tensorflow_serving_api-2.7.0-py2.py3-none-any.whl (37 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━ 559.5/615.3 MB 91.7 MB/s eta 0:00:01
ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device



In [13]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

ImportError: /home/mike/Developer/ml-engineer-zoomcamp/.venv/lib/python3.10/site-packages/tensorflow/python/../libtensorflow_cc.so.2: undefined symbol: _ZN6tflite14ConverterFlags5Impl_66_i_give_permission_to_break_this_code_default_qdq_conversion_mode_E

In [ ]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [ ]:
from keras_image_helper import create_preprocessor


In [ ]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))


In [ ]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [ ]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [ ]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [ ]:
pb_response = stub.Predict(pb_request, timeout=20.0)

preds = pb_response.outputs['dense_7'].float_val

In [ ]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [ ]:
dict(zip(classes, preds))